# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from real biological analysis of our FASTQ files in lectures 8 and 9.

As described in the Jupyter notebook for that lecture, the FASTQ reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

the HA count is  5409
the HA count is  4122

Source: Lecture 8 - TFCB2021

The end of NA is:   CACGATAGATAAATAATAGTGCACCAT

The end of HA is:   CCGGATTTGCATATAATGATGCACCAT

From class
1. Get the reverse complement of each read.
2. Determine if it matches the expected pattern for HA and NA, and if so which one.
3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.

In [10]:
import Bio.SeqIO
import re

Read in the sequencing reads and convert them to strings

In [ ]:
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

Find the reverse complement of the string

In [37]:
def reverse_complement(seq):

    """Get reverse complement of a DNA sequence.
    
    Parameters
    -----------
    seq : str
        Uppercase DNA sequence.
        
    Returns
    -------
    str
        Reverse complement of the sequence in upper case.
        
    Example
    --------
    >>> reverse_complement('ATGCAC')
    'GTGCAT'
    
    """
    rc_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N'}
    revcomplement = []
    for nt in reversed(seq.upper()):
        revcomplement.append(rc_dict[nt])
    return ''.join(revcomplement)

rcseqreads = [reverse_complement(read) for read in seqreads ]

assert rcseqreads[1] == reverse_complement(seqreads[1])

a = 'CACGATAGATAAATAATAGTGCACCAT'
len(a)

27

In [59]:
HAseq = 'CCGGATTTGCATATAATGATGCACCAT'
NAseq = 'CACGATAGATAAATAATAGTGCACCAT'
HAcount = 0
NAcount = 0
for seq in rcseqreads:
    if HAseq in seq:
        HAcount+=1
    elif NAseq in seq:
        NAcount+=1


print('the HA count is ',HAcount)
print('the HA count is ',NAcount)

the HA count is  5409
the HA count is  4122


In [93]:


def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : str
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
    Example
    -------
    >>> read_barcode('TTTTTTTTTTTTTTTTGCGGCCGCCT', bclen=16)
    'AAAAAAAAAAAAAAAA'
        
    """
    barcode_matcher = re.compile(f'^(?P<barcode>[ACGT]{{{bclen}}})' + reverse_complement(upstream))
    m = barcode_matcher.search(seqread)
    if not m:
        return None
    else:
        return reverse_complement(seqread[: bclen])
    
    
# now read sequences and apply function
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))
seqreads_str = [str(seqrecord.seq) for seqrecord in seqreads]

    

# Get the counts of all barcodes
barcode_counts = {}
n_invalid = 0
for seq in seqreads_str:
    barcode = read_barcode(seq, bclen=16)
    if not barcode:
        n_invalid += 1
    elif barcode not in barcode_counts:
        barcode_counts[barcode] = 1
    else:
        barcode_counts[barcode] += 1
        
hareads=[]
nareads=[]
for seq in seqreads_str:
    if HAseq in seqreads_str:
       hareads.append(seq)
    
print(f"Parsed {len(seqreads_str)} sequences, of which {n_invalid} lacked valid barcodes")



Parsed 10000 sequences, of which 569 lacked valid barcodes


0

In [111]:
nareads={}
hareads={}

for seq in rcseqreads:
    if HAseq in seq:
        if seq[-16:] in hareads:
            hareads[seq[-16:]] +=1
        else:
            hareads[seq[-16:]] =1
    elif NAseq in seq:
        if seq[-16:] in nareads:
            nareads[seq[-16:]]+=1
        else:
            nareads[seq[-16:]] =1
    else:
        continue

import pandas

na_barcode_count = pandas.Series(nareads).reset_index()
ha_barcode_count = pandas.Series(hareads).reset_index()

ha_barcode_count.sort_values(0, ascending=False)



,index,0
14,CCCGACCCGACATTAA,157
30,GATTTCCGATCAGTCT,124
4,GCTACTACTATACCAT,121
37,CTCCATCACTCGCCAA,116
41,AACCGTCACCTCAACC,106
...,...,...
203,TGGGCAATAAACGTAG,1
204,TTATACCTTAGTCTGT,1
205,TCTTAACAGCATCAGC,1
206,TTCGACTTCCTAATAC,1


In [110]:
na_barcode_count.sort_values(0, ascending=False)

,index,0
25,ACCAGTTCTCCCCGGG,153
13,TCAAGAAGCCTTGGAG,151
47,TGACGATCCTCAAGAA,143
4,CGTCTTCCATCCCCAT,133
35,TCTGGCAGGCCTCGCT,125
...,...,...
84,GAACCCTAGAAGCCCC,1
83,TGTGCAGCAAAAGTAG,1
81,CACCAGTTCTCCCGGG,1
80,CGTCTTCCATCCGCAT,1


Use a regular expression to find the HA or NA strings

2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

The HA barcode with the most reads is CCCGACCCGACATTAA  with	157 reads


The NA barcode with the most reads is ACCAGTTCTCCCCGGG  with 	153 reads